__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-19_12-39_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-22_09-57_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-26_11-22_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-08_20-38_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-14_15-54_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-01_13-58_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-04_17-12_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-07_10-46_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-15_13-16_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

15169


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

15169


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
5750,2168481692,Участок 6 сот. (ИЖС),2000000,,"Фортификационная ул., 100к100","Продам два смежных участка по 6 соток на ул. Фортификационная. \nКатегория земли: Земли населённых пунктов.\nКадастровые номера: 91:01:008001:317 и 91:01:008001:733 к участкам подведена дорога, в конце лета будет подключено электричество. Цена указана за 6 с",5,2022-09-19 12:39:53.571,sevastopol,6.00,True,2.00,0.33,4-8
11461,2443720738,"Участок 7,7 сот. (ИЖС)",2800000,,"садоводческое товарищество Рыбак-6, 95","Срочно, цена снижена временно! \nПродам земельный участок в Казачьей бухте возле моря! \n\nРасположен в СТ «Рыбак-6» (рядом находятся «Рыбак-7», «Ахтиар», «Скалистый», «Атлантика-1 и 2, и другие кооперативы.\n\nОбщая площадь 7,67 соток.\n\nКадастровый номер 91:02",9,2022-11-01 13:58:37.789,sevastopol,7.70,True,2.80,0.36,4-8
670,2287197200,"Участок 4,5 сот. (ИЖС)",5200000,,Муромская ул.,"Продаю свой участок Ижс 4,5 соток! Первая линия ул. Горпищенко! Адрес ул. Муромская 138/7-А!, кадастровый номер 91:04:000000:870.\nДлина участка 34м, ширина 16м! \nУчасток ровный! \nВ двух минутах остановка, рядом школы, садики, магазины, торговый центр! \nЕст",15,2022-09-08 16:04:40.227,sevastopol,4.50,True,5.20,1.16,4-8


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15169 entries, 0 to 15168
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            15169 non-null  int64         
 1   title               15169 non-null  object        
 2   price               15169 non-null  int64         
 3   obj_name            15169 non-null  object        
 4   adr                 15169 non-null  object        
 5   description         15169 non-null  object        
 6   avito_page          15169 non-null  int64         
 7   ts                  15169 non-null  datetime64[ns]
 8   place               15169 non-null  object        
 9   area                15169 non-null  float64       
 10  is_IJS              15169 non-null  bool          
 11  priceM              15169 non-null  float64       
 12  priceMU             15169 non-null  float64       
 13  area_size_category  15159 non-null  category  

---

In [14]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
# data_cian = data_cian.rename(columns={'cian_url':'url'})

In [15]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

15169


## обновляем таблицу адресов

In [16]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [17]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [18]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/15169 [00:00<?, ?it/s]

  0%|          | 0/15169 [00:00<?, ?it/s]

In [19]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
12519,"Нахимовский муниципальный округ, посёлок Любимовка","Севастополь,Нахимовский район , Любимовка"
4222,коттеджный пос. Сияние Севастополя,"Севастополь,коттеджный пос. Сияние Севастополя"
9333,"садоводческое товарищество Вишенка, ул. Анатолия Дрыгина","Севастополь,садоводческое товарищество Вишенка, ул. Анатолия Дрыгина"
4640,ул. 2-я Линия,"Севастополь,ул. 2-я Линия"
14869,ул. Глухова,"Севастополь,ул. Глухова"
699,"Балаклава, Фонтанный пер., 28А","Севастополь,Балаклава, Фонтанный пер., 28А"
7712,"Балаклава, ул. Василия Жукова","Севастополь,Балаклава, ул. Василия Жукова"
13274,ул. 2-я Линия,"Севастополь,ул. 2-я Линия"
260,"ул. Генерала Провалова, 36","Севастополь,ул. Генерала Провалова, 36"
8959,"Камышовое ш., 24/2","Севастополь,Камышовое ш., 24/2"


In [20]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2022-11-15 13:28:42 | LocationUpdater: 7192 addresses in index
[INFO    ] 2022-11-15 13:28:42 | LocationUpdater: 7240 addresses total
[INFO    ] 2022-11-15 13:28:42 | LocationUpdater: 7192 addresses defined
[INFO    ] 2022-11-15 13:28:42 | LocationUpdater: 48 addresses undefined


  0%|          | 0/48 [00:00<?, ?it/s]

[INFO    ] 2022-11-15 13:28:57 | LocationUpdater: 48 new addresses found


In [21]:
print( len(loc) )
loc.sample(3)

7240


,latitude,longitude,adr
4356,44.57,33.46,"Севастополь,ул. Романтиков, 22"
3267,44.60,33.47,"Севастополь,Коралловая ул., 1"
3152,44.67,33.55,"Севастополь,Фёдоровская ул."


## дополняем данные геометкой

In [22]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

assert len(data)>0
data.to_pickle(result_file_path)

data.info()

всего записей: 15169
записей без геометки: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15169 entries, 0 to 15168
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               15169 non-null  object        
 1   price               15169 non-null  int64         
 2   obj_name            15169 non-null  object        
 3   adr_orig            15169 non-null  object        
 4   description         15169 non-null  object        
 5   avito_page          15169 non-null  int64         
 6   ts                  15169 non-null  datetime64[ns]
 7   place               15169 non-null  object        
 8   area                15169 non-null  float64       
 9   is_IJS              15169 non-null  bool          
 10  priceM              15169 non-null  float64       
 11  priceMU             15169 non-null  float64       
 12  area_size_category  15159 non-null  category      
 13  u